# P3

In [12]:
import pandas as pd
import numpy as np

**Load the data**

In [22]:
data_body = pd.read_csv('soc-redditHyperlinks-body.tsv', sep = '\t')
data_title = pd.read_csv('soc-redditHyperlinks-title.tsv', sep = '\t')
data_body.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0...."


In [27]:
data_title.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,rddtgaming,rddtrust,1u4pzzs,2013-12-31 16:39:18,1,"25.0,23.0,0.76,0.0,0.44,0.12,0.12,4.0,4.0,0.0,..."
1,xboxone,battlefield_4,1u4tmfs,2013-12-31 17:59:11,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
2,ps4,battlefield_4,1u4tmos,2013-12-31 17:59:40,1,"100.0,88.0,0.78,0.02,0.08,0.13,0.07,16.0,16.0,..."
3,fitnesscirclejerk,leangains,1u50xfs,2013-12-31 19:01:56,1,"49.0,43.0,0.775510204082,0.0,0.265306122449,0...."
4,fitnesscirclejerk,lifeprotips,1u51nps,2013-12-31 21:02:28,1,"14.0,14.0,0.785714285714,0.0,0.428571428571,0...."


**Create the dataframe containing body + title links**

In [35]:
data_total = pd.concat([data_body,data_title], ignore_index = True)
data_total.shape

(858488, 6)

**Count the number of nodes and edges**

In [36]:
table = pd.DataFrame(columns = ['body', 'title', 'total'], index = ['Nodes', 'Edges', '+ edges', '- edges', 'Triads'])

nb_edges_body = data_body.shape[0]
nb_nodes_body = np.unique(np.concatenate((data_body['SOURCE_SUBREDDIT'].values , data_body['TARGET_SUBREDDIT'].values))).shape[0]

nb_edges_title = data_title.shape[0]
nb_nodes_title = np.unique(np.concatenate((data_title['SOURCE_SUBREDDIT'].values , data_title['TARGET_SUBREDDIT'].values))).shape[0]

nb_edges_total = data_total.shape[0]
nb_nodes_total = np.unique(np.concatenate((data_total['SOURCE_SUBREDDIT'].values , data_total['TARGET_SUBREDDIT'].values))).shape[0]


table.loc['Nodes'] = [nb_nodes_body, nb_nodes_title, nb_nodes_total]
table.loc['Edges'] = [nb_edges_body, nb_edges_title, nb_edges_total]

table.head()

,body,title,total
Nodes,35776,54075,67180
Edges,286561,571927,858488
+ edges,NaN,NaN,NaN
- edges,NaN,NaN,NaN
Triads,NaN,NaN,NaN


**Count the proprtion of positives edges**

In [37]:
#Number of positive (negative) edges :
#We first filter the data by keeping only +1 (or -1) values and then we find the number of lines (edges) that this filtering gives
nb_pos_edges_body = data_body[data_body['LINK_SENTIMENT']==1].shape[0] 
nb_neg_edges_body = data_body[data_body['LINK_SENTIMENT']==-1].shape[0]

nb_pos_edges_title = data_title[data_title['LINK_SENTIMENT']==1].shape[0] 
nb_neg_edges_title= data_title[data_title['LINK_SENTIMENT']==-1].shape[0]

nb_pos_edges_total = data_total[data_total['LINK_SENTIMENT']==1].shape[0] 
nb_neg_edges_total = data_total[data_total['LINK_SENTIMENT']==-1].shape[0]

#Proportion of positive/negative edges:
#We divide the number of positive (or negative) edges previouly found and divide it by the total number of edges
#We use the table1 to find the total number of edges as we already filled these informations
perc_pos_body = nb_pos_edges_body / table.loc['Edges']['body']
perc_neg_body = nb_neg_edges_body / table.loc['Edges']['body']

perc_pos_title = nb_pos_edges_title / table.loc['Edges']['title']
perc_neg_title = nb_neg_edges_title / table.loc['Edges']['title']

perc_pos_total = nb_pos_edges_total / table.loc['Edges']['total']
perc_neg_total = nb_neg_edges_total / table.loc['Edges']['total']

In [38]:
#We must indicate a percentage format : we use the {:.1%} format to have a percentage with only one decimal, as in the paper 
table.loc['+ edges'] = ['{:.1%}'.format(perc_pos_body), '{:.1%}'.format(perc_pos_title), '{:.1%}'.format(perc_pos_total)]
table.loc['- edges'] = ['{:.1%}'.format(perc_neg_body), '{:.1%}'.format(perc_neg_title), '{:.1%}'.format(perc_neg_total)]

table.head()

,body,title,total
Nodes,35776,54075,67180
Edges,286561,571927,858488
+ edges,92.6%,89.3%,90.4%
- edges,7.4%,10.7%,9.6%
Triads,NaN,NaN,NaN


In [40]:
table['remove'] = [1,2,3,0,0]
table.head()

,body,title,total,remove
Nodes,35776,54075,67180,1
Edges,286561,571927,858488,2
+ edges,92.6%,89.3%,90.4%,3
- edges,7.4%,10.7%,9.6%,0
Triads,NaN,NaN,NaN,0


**Remove some lines with positive link to have 80%**

In [44]:
data_remove = pd.concat((data_total[data_total['LINK_SENTIMENT']==-1], data_total[data_total['LINK_SENTIMENT']==1].head(329660)))

nb_edges_remove = data_remove.shape[0]
nb_nodes_remove = np.unique(np.concatenate((data_remove['SOURCE_SUBREDDIT'].values , data_remove['TARGET_SUBREDDIT'].values))).shape[0]

nb_pos_edges_remove = data_remove[data_remove['LINK_SENTIMENT']==1].shape[0] 
nb_neg_edges_remove = data_remove[data_remove['LINK_SENTIMENT']==-1].shape[0]

perc_pos_remove = nb_pos_edges_remove / nb_edges_remove
perc_neg_remove = nb_neg_edges_remove / nb_edges_remove

table['remove'] = [nb_nodes_remove, nb_edges_remove, '{:.1%}'.format(perc_pos_remove), '{:.1%}'.format(perc_neg_remove), 0]
table.head()

,body,title,total,remove
Nodes,35776,54075,67180,42809
Edges,286561,571927,858488,411870
+ edges,92.6%,89.3%,90.4%,80.0%
- edges,7.4%,10.7%,9.6%,20.0%
Triads,NaN,NaN,NaN,0
